In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install ipython-sql
!pip install pandas
!pip install pandasql
!pip install geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
%cd /content/drive/MyDrive/DivvyBike/

/content/drive/MyDrive/DivvyBike


In [4]:
import pandas as pd

# List of file names
file_names = [
    '202304-divvy-tripdata.csv',
    '202303-divvy-tripdata.csv',
    '202302-divvy-tripdata.csv',
    '202301-divvy-tripdata.csv',
    '202212-divvy-tripdata.csv',
    '202211-divvy-tripdata.csv',
    '202210-divvy-tripdata.csv',
    '202209-divvy-publictripdata.csv',
    '202208-divvy-tripdata.csv',
    '202207-divvy-tripdata.csv',
    '202206-divvy-tripdata.csv',
    '202205-divvy-tripdata.csv'
]

# Specify the file path for the output CSV file
output_file_path = 'combined_divvy_rides.csv'

# Define the column names
column_names = ['ride_id', 'rideable_type', 'started_at', 'ended_at',
                'start_station_name', 'start_station_id', 'end_station_name',
                'end_station_id', 'start_lat', 'start_lng', 'end_lat',
                'end_lng', 'member_casual']

# Chunk size for processing
chunk_size = 100000

# Initialize an empty DataFrame for storing the combined data
df_combined = pd.DataFrame()

# Process each file in chunks and append to the combined DataFrame
for i, file_name in enumerate(file_names):
    file_path = file_name
    
    # Read the file in chunks
    for chunk in pd.read_csv(file_path, header=None, chunksize=chunk_size, engine='python'):
        df_combined = pd.concat([df_combined, chunk])

# Save the combined DataFrame to the output CSV file without column names
df_combined.to_csv(output_file_path, index=False, header=False)

In [5]:
import pandas as pd

# Read the CSV file with specified data types or low_memory=False
df_combined = pd.read_csv('combined_divvy_rides.csv', dtype={'column_name': str}, low_memory=False)

# Print the first row
print(df_combined.head(1))

            ride_id  rideable_type           started_at             ended_at  \
0  8FE8F7D9C10E88C7  electric_bike  2023-04-02 08:37:28  2023-04-02 08:41:37   

  start_station_name start_station_id end_station_name end_station_id  \
0                NaN              NaN              NaN            NaN   

  start_lat start_lng end_lat end_lng member_casual  
0      41.8     -87.6   41.79   -87.6        member  


In [6]:
import pandas as pd

# Read the combined CSV file
df_combined = pd.read_csv('combined_divvy_rides.csv')

# Filter rows where rides only started at a station
started_only_count = df_combined[df_combined['end_station_id'].isna() & df_combined['start_station_id'].notna()].shape[0]

# Filter rows where rides only ended at a station
ended_only_count = df_combined[df_combined['start_station_id'].isna() & df_combined['end_station_id'].notna()].shape[0]

# Filter rows where rides both started and ended at a station
started_and_ended_count = df_combined[df_combined['start_station_id'].notna() & df_combined['end_station_id'].notna()].shape[0]

# Calculate the total number of rides
total_rides = len(df_combined)

# Calculate the percentages
started_only_percentage = (started_only_count / total_rides) * 100
ended_only_percentage = (ended_only_count / total_rides) * 100
started_and_ended_percentage = (started_and_ended_count / total_rides) * 100
neither_started_nor_ended_percentage = 100 - started_only_percentage - ended_only_percentage - started_and_ended_percentage

# Print the results
print(f"Percentage of rides that only started at a station: {started_only_percentage:.2f}%")
print(f"Percentage of rides that only ended at a station: {ended_only_percentage:.2f}%")
print(f"Percentage of rides that both started and ended at a station: {started_and_ended_percentage:.2f}%")
print(f"Percentage of rides that neither started nor ended at a station: {neither_started_nor_ended_percentage:.2f}%")


<ipython-input-6-67e64d3c9857>:4: DtypeWarning: Columns (8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_combined = pd.read_csv('combined_divvy_rides.csv')


Percentage of rides that only started at a station: 8.41%
Percentage of rides that only ended at a station: 7.43%
Percentage of rides that both started and ended at a station: 77.38%
Percentage of rides that neither started nor ended at a station: 6.77%


In [7]:
#Make a visualization of the most frequent Divvy bike locations
import pandas as pd
import folium
from folium.plugins import HeatMap

# Read the combined Divvy rides data
df_combined = pd.read_csv('combined_divvy_rides.csv')

# Group the data by start station coordinates and count the number of rides
station_counts = df_combined.groupby(['start_lat', 'start_lng']).size().reset_index(name='ride_count')

# Sort the stations by ride count in descending order
station_counts = station_counts.sort_values('ride_count', ascending=False)

# Take the top N stations with the highest ride count
top_n_stations = station_counts.head(10)

# Create a folium map centered around Chicago
map_divvy = folium.Map(location=[41.8781, -87.6298], zoom_start=11)

# Add a heatmap layer based on the start station coordinates and ride count
heatmap_data = top_n_stations[['start_lat', 'start_lng', 'ride_count']].values.tolist()
HeatMap(heatmap_data).add_to(map_divvy)

# Add markers for the top N stations
for idx, row in top_n_stations.iterrows():
    lat, lng = row['start_lat'], row['start_lng']
    ride_count = row['ride_count']
    folium.Marker(
        location=[lat, lng],
        popup=f"Ride Count: {ride_count}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map_divvy)

# Display the map
map_divvy

<ipython-input-7-a45f21aa7fbc>:7: DtypeWarning: Columns (8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_combined = pd.read_csv('combined_divvy_rides.csv')


In [8]:
import pandas as pd
from scipy.spatial import distance
import numpy as np
import time
from tqdm import tqdm

# Read the Divvy Bike stations data
df_stations = pd.read_csv('Divvy_Bicycle_Stations_-_All_-_Map.csv')

# Create a HashMap of station coordinates and names
station_map = dict(zip(zip(df_stations['Latitude'], df_stations['Longitude']), df_stations['Station Name']))

# Load the combined bike rides data in chunks
chunk_size = 100000
ride_chunks = pd.read_csv('combined_divvy_rides.csv', chunksize=chunk_size)

# Prepare a list to store filtered data
filtered_data = []

# Iterate over the ride chunks
for df_chunk in ride_chunks:
    # Filter rides without Divvy bike start or end station
    df_filtered = df_chunk[df_chunk['start_station_id'].isna() & df_chunk['end_station_id'].isna()]

    # Filter out rows with missing or invalid coordinates
    df_filtered = df_filtered.dropna(subset=['start_lat', 'start_lng'])

    # Convert coordinates to float
    df_filtered['start_lat'] = df_filtered['start_lat'].astype(float)
    df_filtered['start_lng'] = df_filtered['start_lng'].astype(float)

    # Calculate distances using the HashMap
    ride_start_points = df_filtered[['start_lat', 'start_lng']].values
    nearest_station_coords = list(station_map.keys())
    distances = distance.cdist(ride_start_points, nearest_station_coords, metric='cityblock')
    nearest_station_indices = distances.argmin(axis=1)
    nearest_station_names = [station_map[nearest_station_coords[i]] for i in nearest_station_indices]

    # Add the filtered data to the list
    df_filtered['nearest_station_distance'] = distances.min(axis=1)
    df_filtered['nearest_station_name'] = nearest_station_names
    filtered_data.append(df_filtered)

# Concatenate all filtered data into a single DataFrame
df_filtered = pd.concat(filtered_data)

# Select the required columns
columns_to_keep = ['nearest_station_distance', 'nearest_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng']
df_filtered = df_filtered[columns_to_keep]

# Save the filtered data to a new file
df_filtered.to_csv('binned_nearest_divvy_location.csv', index=False)

# Method 2: Preparation Stage
start = time.time()

# Define the values to bin
values = df_filtered['nearest_station_distance'].values

# Set the number of bins
bins = 100000

# Calculate bin intervals
values_min = values.min()
values_max = values.max()
values_range = values_max - values_min
interval_range = values_range / bins

bin_intervals = {b: (values_min + b * interval_range, values_min + (b + 1) * interval_range) for b in range(bins)}

# Fill up bins with indices of values
bin_idx_dict = {}
for b, interval in tqdm(bin_intervals.items()):
    low = interval[0]
    high = interval[1]
    bin_idx_dict[b] = np.where((values >= low) & (values < high))[0].flatten()

end = time.time()
print(end - start)

# Method 2: Implementation
start = time.time()

# Define the value to bin mapping function
val_to_bin_idx = lambda x: int((x - values_min) / interval_range)

# Define the value for which to find the nearest neighbors
val_match = values.min()  # Minimum distance between the bike and any of the stations

neighbor_bin_idx_center = val_to_bin_idx(val_match)
neighbor_center_idx = bin_idx_dict[neighbor_bin_idx_center]
neighbor_center = values[neighbor_center_idx]

neighbor_left = None
neighbor_bin_idx_left = neighbor_bin_idx_center
while neighbor_left is None:
    neighbor_bin_idx_left -= 1
    try:
        neighbor_left_idx = bin_idx_dict[neighbor_bin_idx_left]
        if neighbor_left_idx.shape[0] > 0:
            neighbor_left = values[neighbor_left_idx]
    except KeyError:
        neighbor_left_idx = np.array([])
        neighbor_left = np.array([])

neighbor_right = None
neighbor_bin_idx_right = neighbor_bin_idx_center
while neighbor_right is None:
    neighbor_bin_idx_right += 1
    try:
        neighbor_right_idx = bin_idx_dict[neighbor_bin_idx_right]
        if neighbor_right_idx.shape[0] > 0:
            neighbor_right = values[neighbor_right_idx]
    except KeyError:
        neighbor_right_idx = np.array([])
        neighbor_right = np.array([])

neighbor_idx = np.concatenate([neighbor_left_idx, neighbor_center_idx, neighbor_right_idx]).astype(int)
neighbors = np.concatenate([neighbor_left, neighbor_center, neighbor_right])

dist1d = np.abs(neighbors - val_match)
min_dist = dist1d.min()
idx = np.where(dist1d == min_dist)[0]  # Fix the indexing error here
print(val_match)
print(neighbor_idx[idx], values[neighbor_idx[idx]])

end = time.time()
print(end - start)



<ipython-input-8-8b8ba2dea8be>:21: DtypeWarning: Columns (8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  for df_chunk in ride_chunks:
<ipython-input-8-8b8ba2dea8be>:21: DtypeWarning: Columns (8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  for df_chunk in ride_chunks:
<ipython-input-8-8b8ba2dea8be>:21: DtypeWarning: Columns (8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  for df_chunk in ride_chunks:
<ipython-input-8-8b8ba2dea8be>:21: DtypeWarning: Columns (8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  for df_chunk in ride_chunks:
<ipython-input-8-8b8ba2dea8be>:21: DtypeWarning: Columns (8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  for df_chunk in ride_chunks:
<ipython-input-8-8b8ba2dea8be>:21: DtypeWarning: Columns (8,9,10,11) have mixed types. Specify dtype option on import or set low_m

51.987762212753296
0.00021518799999142857
[   108    518    770 ... 396703 396706 396750] [0.00021519 0.00021519 0.00021519 ... 0.00021519 0.00021519 0.00021519]
0.006096839904785156


In [9]:
import pandas as pd
import folium
from folium.plugins import HeatMap

# Read the filtered data with nearest Divvy bike locations
df_filtered = pd.read_csv('binned_nearest_divvy_location.csv')

# Group the data by nearest station coordinates and count the number of occurrences
station_counts = df_filtered.groupby(['start_lat', 'start_lng']).size().reset_index(name='count')

# Sort the stations by count in descending order
station_counts = station_counts.sort_values('count', ascending=False)

# Take the top N stations with the highest count
top_n_stations = station_counts.head(10)

# Create a folium map centered around Chicago
map_divvy = folium.Map(location=[41.8781, -87.6298], zoom_start=11)

# Add a heatmap layer based on the start station coordinates and count
heatmap_data = top_n_stations[['start_lat', 'start_lng', 'count']].values.tolist()
HeatMap(heatmap_data).add_to(map_divvy)

# Add markers for the top N stations
for idx, row in top_n_stations.iterrows():
    lat, lng = row['start_lat'], row['start_lng']
    count = row['count']
    folium.Marker(
        location=[lat, lng],
        popup=f"Count: {count}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map_divvy)

# Display the map
map_divvy
